In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
# import bert
# from bert import BertModelLayer
import functools

from tensorflow import keras
import tensorflow as tf
import os
import sentencepiece as spm

import tf_sentencepiece as tfs
import tensorflow_text as tftext
import sys
sys.path.extend(["/space/users/jgeorge/git/tensorflow_repos/models"])


import json
import math
import os
import csv
import collections
import pandas as pd

from absl import app
from absl import flags
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub

# pylint: disable=g-import-not-at-top,redefined-outer-name,reimported
from official.modeling import model_training_utils

from official.nlp.modeling.models import bert_classifier, bert_pretrainer
# from official.nlp.modeling.models.bert_classifier import BertClassifier
# from official.nlp.modeling.models.bert_pretrainer import BertPretrainer
# from official.nlp import bert_modeling as modeling
# from official.nlp import bert_models
from official.nlp import optimization
from official.nlp.bert import common_flags
from official.nlp.bert import input_pipeline
from official.nlp.bert import model_saving_utils
from official.utils.misc import distribution_utils
from official.utils.misc import keras_utils
from official.nlp.bert import tokenization

from official.nlp.albert import configs as albert_configs
from official.nlp.bert import run_classifier as run_classifier_bert

from tensorflow.python.keras.utils import tf_utils
from sklearn.model_selection import train_test_split

In [8]:
from tensorflow.python.keras.engine import network

In [9]:
FLAGS = flags.FLAGS    

In [10]:
print(tf.__version__)

2.2.0


In [11]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

In [12]:
##restricting no of gpus
gpus = tf.config.experimental.list_physical_devices('GPU')
device_to_use = gpus[3]
tf.config.experimental.set_memory_growth(device_to_use,True)
tf.config.experimental.set_visible_devices(device_to_use, 'GPU')
print(tf.config.experimental.get_visible_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]


In [13]:
##setting direcory for downloading tfhub modules
os.environ["TFHUB_CACHE_DIR"] = '/space/engineering/tfhub_modules'

In [14]:
dish_data_path='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/dishDataJan17.txt'
df = pd.read_csv(dish_data_path,sep='\t',header=None,names=['filename','text','granular_intent','ru_intent'])

In [15]:
df.head()

,filename,text,granular_intent,ru_intent
0,INT-sv1appis14-1504137880316-305603_4567,can you send my bill to my mail?,billing-preferences,billing-preferences
1,INT-va1appis15-1504373018548-41332,My Wally receiver has lost Satellite signal in...,comp_part_signal_loss-issue,comp_part_signal_loss-issue
2,INT-sv1appis12-1503954587819-263368,I need a payment extension so i don't get my s...,payment_extension-request,payment_extension-request
3,8ed8e9b1-10f5-438d-c409-e616c3ff9ede,how can i find my local channels. it seems i d...,channel_package-issue,channel_package-issue
4,INT-sv1appis13-1504099917638-293735,Wanted to speak with someone about my bill,representative-request,representative-request


In [16]:
df_train,df_val,y_train,y_val = train_test_split(df,df['granular_intent'],train_size=0.8,random_state=42,stratify=df['granular_intent'])

In [17]:
# main_exp_folder = '/space/users/jgeorge/projects/k/tensorflow2-question-answering/input/dish/data/jan17_2020/'
main_exp_folder = '/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/'

In [18]:
output_folder = '/var/extra/users/jgeorge/tf2.0/input/dish/models/albert_en_large'

In [13]:
train_csv_file='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/train.txt'
eval_csv_file='/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/val.txt'

df_train.to_csv(train_csv_file,sep='\t',header=False,index=False)
df_val.to_csv(eval_csv_file,sep='\t',header=False,index=False)

In [19]:
intent_file = '/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/intentlist.txt'

In [20]:
with open(intent_file,'w',encoding='utf-8') as out_f:
    for intent in sorted(df_train['granular_intent'].unique()):
        out_f.write(intent+'\n')


In [21]:
intent_list = []
with open(intent_file,'r') as inp_f:
    for intent in inp_f:
        intent_list.append(intent.strip())

In [22]:
input_meta_data = {
    'max_seq_length':128,
    'num_labels':len(intent_list),
    'train_data_size':20574,
    'eval_data_size':5144,
    'default_label':'other-other'
}

In [23]:
input_meta_data

{'max_seq_length': 128,
 'num_labels': 142,
 'train_data_size': 20574,
 'eval_data_size': 5144,
 'default_label': 'other-other'}

In [24]:
input_meta_path = os.path.join(main_exp_folder,'input_metadata.json')

In [25]:
#writing metadata file
with open(input_meta_path,'w',encoding='utf-8') as jf:
    jf.write(json.dumps(input_meta_data))

In [26]:
#reading metadata file
with open(input_meta_path,'r',encoding='utf-8') as jf:
#     input_meta_data = json.loads(jf.read())
    input_meta_data = json.load(jf)

## Data preprocessing & feeding into graph
We will be reading the regular text file as input & encoding the text 
using sentence piece encoder provided along with albert & then
writing it as tfrecord. Other than the sentencepiece encoder these models does not require any preprocessing like regex-replacement, lemmatization etc 

Writing as tfrecord is not necessary, but it is a more optimized file format for reading into 
tf.data api (you can even base your tf dataset on a textfile or even a python iterator)
Also it's not necessary to use tf.data apis but this is much more optimized, like preloading data into gpu memory & optimization required if you are running across systems & all

### Sentence piece encoding

In [27]:
##albert hub url
bert_hub_url='https://tfhub.dev/tensorflow/albert_en_large/1'

In [28]:
# download from link given in https://github.com/tensorflow/models/tree/master/official/nlp/albert
# specifically https://storage.googleapis.com/cloud-tpu-checkpoints/albert/checkpoints/albert_v2_large.tar.gz
# this is not required if we are just using tensorflow hub, since the hub module didn't have a detailed config, 
# will load the config from this folder for reference  
bert_model_dir='/space/engineering/pretrained_models/albert/albert_large'

We need to load the used sentence piece model from the hub module, 
it will be present in the assests folder of the downloaded albert model
Since we are using hub here & not directly downloading it, the path can be fetched from hub layer

In [102]:
multilingual_vocab_path = "/space/engineering/pretrained_models/bert/multi_cased_L-12_H-768_A-12/vocab.txt"

In [ ]:
tfs.encode()

In [29]:
bert_hub_loaded = hub.load(bert_hub_url)
sentencepiece_path = bert_hub_loaded.sp_model_file.asset_path.numpy()

# or if you are using hub.KeraLayer
# bert_model = hub.KerasLayer(bert_hub_url,trainable=True)
# sentencepiece_path = bert_model.resolved_object.sp_model_file.asset_path.numpy()

##if you have directly downloaded the model you could do this
# sentencepiece_path = os.path.join(model_dir, "assets", "30k-clean.model")



In [30]:
##loading the sentence piece model into memory
sp_model_proto = tf.io.gfile.GFile(sentencepiece_path, 'rb').read()

# sp_model = sp_model_proto

For sentence piece we can use either the       
1) tf_sentencepiece (import as tfs here)     
2) sentencepiece (imported as spm) package        
3) sentencepiece from tensorflow_text    
4) tokenization.FullSentencePieceTokenizer - a wrapper on top of 2) sentencepiece, code is available in models/official/nlp/bert/tokenization.py      

We will eventually just use tf_sentencepiece as that's the one which is currently, available to export as a graph.      
But for the 1st step of creating training data (in tf records format) either of them will do
but i'll illustrate how to use others as well

In [36]:
encoded_text3

<tf.Tensor: shape=(16,), dtype=int32, numpy=
array([  590,    55,  7876,    20,  6557,  3284,  5477, 11969,   357,
           8,  3099,     8,  1323,  1433,   159, 10114], dtype=int32)>

In [33]:
actual_sentence = 'give me directions to nearest restaurant randomtext 15-08-2019 year 2019'
#1st method
encoded_text1 = tfs.encode([actual_sentence],model_proto=sp_model_proto)
##This method return
#   pieces: A dense 2D tensor representing the tokenized sentences. key = values
#   sequence_length: A 1D tensor representing the length of pieces.

encoded_value1 = encoded_text1.values.numpy()
sequence_length=encoded_text1.sequence_length.numpy()[0]
# sequence_length=len(encoded_value1[0])
print('encoded_text ',encoded_text1, ', decoded text ',
      tfs.decode(encoded_value1,sequence_length=[sequence_length],model_proto=sp_model_proto),'\n')

# 2nd method
sp2 = spm.SentencePieceProcessor()
sp2.load(model_proto=sp_model_proto)
encoded_text2 = sp2.encode_as_ids(actual_sentence)
print('encoded_text2 ',encoded_text2, ', decoded text2 ',
      sp2.decode_ids(encoded_text2),'\n')

###3rd methond
sp3 = tftext.SentencepieceTokenizer(model=sp_model_proto)
encoded_text3 = sp3.tokenize(actual_sentence)
print('encoded_text3 ',encoded_text3, ', decoded text3 ',sp3.detokenize(encoded_text3.numpy()),'\n')


#method 4
#this is just another wrapper around sentencepiece
sp4 = tokenization.FullSentencePieceTokenizer(sentencepiece_path)
tokens4= sp4.tokenize(actual_sentence)
ids4 = sp4.convert_tokens_to_ids(tokens4)
print('tokens4',tokens4, '\nids4 ',ids4)

encoded_text  SentencepieceEncodeDense(values=<tf.Tensor: shape=(1, 16), dtype=int32, numpy=
array([[  590,    55,  7876,    20,  6557,  3284,  5477, 11969,   357,
            8,  3099,     8,  1323,  1433,   159, 10114]], dtype=int32)>, sequence_length=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([16], dtype=int32)>) , decoded text  tf.Tensor([b'give me directions to nearest restaurant randomtext 15-08-2019 year 2019'], shape=(1,), dtype=string) 

encoded_text2  [590, 55, 7876, 20, 6557, 3284, 5477, 11969, 357, 8, 3099, 8, 1323, 1433, 159, 10114] , decoded text2  give me directions to nearest restaurant randomtext 15-08-2019 year 2019 

encoded_text3  tf.Tensor(
[  590    55  7876    20  6557  3284  5477 11969   357     8  3099     8
  1323  1433   159 10114], shape=(16,), dtype=int32) , decoded text3  tf.Tensor(b'give me directions to nearest restaurant randomtext 15-08-2019 year 2019', shape=(), dtype=string) 

tokens4 ['▁give', '▁me', '▁directions', '▁to', '▁nearest', '▁restaur

You would have noticed that in method 1) tf_sentencepiece i have to pass sequence_length for decoding
Printing output from couple of other functions in sentencepiece, just to get a better understanding

In [32]:
encoded_text1

SentencepieceEncodeDense(values=<tf.Tensor: shape=(1, 16), dtype=int32, numpy=
array([[  590,    55,  7876,    20,  6557,  3284,  5477, 11969,   357,
            8,  3099,     8,  1323,  1433,   159, 10114]], dtype=int32)>, sequence_length=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([16], dtype=int32)>)

In [99]:
tfs.encode([actual_sentence,'sentence test'],model_proto=sp_model_proto)

SentencepieceEncodeDense(values=<tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  590,    55,  7876,    20,  6557,  3284,  5477, 11969,   357,
            8,  3099,     8,  1323,  1433,   159, 10114],
       [ 5123,  1289,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, sequence_length=<tf.Tensor: shape=(2,), dtype=int32, numpy=array([16,  2], dtype=int32)>)

In [27]:
sp3.tokenize(['hi this is','test run'])

<tf.RaggedTensor [[4148, 48, 25], [1289, 485]]>

In [28]:
actual_sentence

'give me directions to nearest restaurant randomtext 15-08-2019 year 2019'

In [29]:
pieces1 = sp2.encode_as_pieces(actual_sentence)
ids1 = sp2.piece_to_id(pieces1)
print('pieces1 ', pieces1 , '\nids1 ',ids1,'\n')

pieces2 = sp2.encode_as_pieces('give me directions to nearestrestaurant')
ids2 = sp2.piece_to_id(pieces2)
pieces2_2 = sp2.id_to_piece(ids2)
decoded_piece = sp2.decode_pieces(pieces2)

print('pieces2 ', pieces2, '\nids2 ',ids2,'\npieces2_2 ',pieces2_2, '\ndecoded_piece ',decoded_piece )


pieces1  ['▁give', '▁me', '▁directions', '▁to', '▁nearest', '▁restaurant', '▁random', 'text', '▁15', '-', '08', '-', '20', '19', '▁year', '▁2019'] 
ids1  [590, 55, 7876, 20, 6557, 3284, 5477, 11969, 357, 8, 3099, 8, 1323, 1433, 159, 10114] 

pieces2  ['▁give', '▁me', '▁directions', '▁to', '▁nearest', 'rest', 'au', 'rant'] 
ids2  [590, 55, 7876, 20, 6557, 11466, 1346, 7874] 
pieces2_2  ['▁give', '▁me', '▁directions', '▁to', '▁nearest', 'rest', 'au', 'rant'] 
decoded_piece  give me directions to nearestrestaurant


In [30]:
sp2.id_to_piece([0,1,2])

['<pad>', '<unk>', '[CLS]']

In [31]:
#printing out 1st 100 tokens, just to get a feel of what all entries are present
for index,token in enumerate (sp2.id_to_piece([*range(100)])):
    print(index,token)

0 <pad>
1 <unk>
2 [CLS]
3 [SEP]
4 [MASK]
5 (
6 )
7 "
8 -
9 .
10 –
11 £
12 €
13 ▁
14 ▁the
15 ,
16 ▁of
17 ▁and
18 s
19 ▁in
20 ▁to
21 ▁a
22 '
23 ▁was
24 ▁he
25 ▁is
26 ▁for
27 ▁on
28 ▁as
29 ▁with
30 ▁that
31 ▁i
32 ▁it
33 ▁his
34 ▁by
35 ▁at
36 ▁her
37 ▁from
38 t
39 ▁she
40 ▁an
41 ▁had
42 ▁you
43 d
44 ▁be
45 :
46 ▁were
47 ▁but
48 ▁this
49 i
50 ▁are
51 ▁my
52 ▁not
53 ▁one
54 ▁or
55 ▁me
56 ▁which
57 ▁have
58 a
59 ▁they
60 ?
61 ▁him
62 e
63 ▁has
64 ▁first
65 ▁all
66 ▁their
67 ▁also
68 ing
69 ed
70 ▁out
71 ▁up
72 ▁who
73 ;
74 ▁been
75 ▁after
76 ▁when
77 ▁into
78 ▁new
79 m
80 ▁there
81 ▁two
82 ▁its
83 ▁would
84 ▁over
85 ▁time
86 ▁so
87 ▁said
88 ▁about
89 ▁other
90 ▁no
91 ▁more
92 ▁can
93 y
94 ▁then
95 ▁we
96 th
97 ▁back
98 ▁what
99 re


### Converting the data into TFRecords
For following functionalities refer the file  - models/official/nlp/data/classifier_data_lib.py       
This includes even the sentence piece tokenization part
#### Defining an object for the 1 row of text input data

In [32]:

#in our case we are doing just classification so we only need 1 text value = text_a
class InputExample(object):
    def __init__(self,uuid,text_a,text_b=None,label=None):
        """Constructs a InputExample.
        Args:
          uuid: Unique id for the example.
          text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
          text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
          label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.uuid = uuid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


#### defining an object for the 1 row of input feature

In [33]:
### in the official one there was something called is_real_example as feature, here i skipped it since it's 
#only classification
class InputFeatures(object):
    def __init__(self,input_ids,input_mask,
                segment_ids,label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


#### functions for converting values to train features so that it can be saved

In [34]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    print("type constant ",type(tf.constant(0)))
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(values):
    """Returns an int64_list from a bool / enum / int / uint.
      #Note that the Feature requires a list as input"""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

In [35]:
train_data_path=os.path.join(main_exp_folder,'train.tfrecords')
eval_data_path=os.path.join(main_exp_folder,'eval.tfrecords')


In [36]:
#special tokens
[CLS_ID,SEP_ID]  = tfs.piece_to_id(['[CLS]','[SEP]'],model_proto=sp_model_proto).numpy()
print('CLS_ID ', CLS_ID, ', SEP_ID ', SEP_ID)

CLS_ID  2 , SEP_ID  3


#### Function which converts a single input to Features, one of the feature being the sentencepiece encoded array

In [37]:
def convert_single_example(ex_index, example, label_list, max_seq_length,sp_proto):
    ## please refer to functionality in file models/official/nlp/data/classifier_data_lib.py 
    ##this function is based on an older version & slightly different as in it's taking care of only 
    # the classification part
    
    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    
#     CLS_ID = 2
#     SEP_ID = 3
    [CLS_ID,SEP_ID]  = tfs.piece_to_id(['[CLS]','[SEP]'],model_proto=sp_proto).numpy()
    # -2 for [CLS], [SEP] 
    # we need to add these tokens in addition to the sentence words is while keeping the max sequence length limit
    # check Bert paper for this [CLS] is added at the beginning & [SEP] after 1st sentence (in our classificatin example 
    # we don't differentiate the sentences here, so [SEP] is at the end. For challenges like SQUAD 2 sentences is 
    # where i have seen people use 2 sentences , 1st question & 2nd answer )
    
    max_word_ids = max_seq_length -2
    input_ids = []
    segment_ids = []
    input_mask = []
    input_ids.append(CLS_ID)
    
    word_ids = tfs.encode([example.text_a],model_proto=sp_proto).values[0]
    
#     print('word ids ',word_ids)
# trimming to the max size
    word_ids = word_ids[:max_word_ids]
    input_ids.extend(word_ids)
#     print("input text ",example.text)
    input_ids.append(SEP_ID)
    segment_ids = [0]*len(input_ids)
    input_mask = [1]*len(input_ids)
    if len(input_ids)<max_seq_length:
        diff = max_seq_length - len(input_ids)
        input_ids.extend([0]*diff)
        segment_ids.extend([0]*diff)
        input_mask.extend([0]*diff)
    assert(len(input_ids)==max_seq_length)
    assert(len(segment_ids)==max_seq_length)
    assert(len(input_mask)==max_seq_length)
    label_map = {label:i for i,label in enumerate(label_list)}
    label_id = label_map[example.label]
    if ex_index < 5:
        logging.info("*** Example ***")
        logging.info("uuid: %s", (example.uuid))
        logging.info("tokens: %s",
                     " ".join([str(x) for x in tfs.id_to_piece(input_ids,model_proto=sp_proto).numpy()]))
        logging.info("input_ids: %s", " ".join([str(x) for x in input_ids]))
        logging.info("input_mask: %s", " ".join([str(x) for x in input_mask]))
        logging.info("segment_ids: %s", " ".join([str(x) for x in segment_ids]))
        logging.info("label: %s (id = %d)", example.label, label_id)

    feature = InputFeatures(input_ids,input_mask,segment_ids,label_id)
    return feature


In [38]:
# bert_config_dict = {
#     "hidden_dropout_prob":0,
#     'initializer_range': 0.02
# }
# bert_config = albert_configs.AlbertConfig.from_dict(bert_config_dict)

# in some cases it might be inside assets folder
bert_config_file = os.path.join(bert_model_dir,'albert_config.json')

# bert_config = modeling.AlbertConfig.from_json_file(FLAGS.bert_config_file)
bert_config = albert_configs.AlbertConfig.from_json_file(bert_config_file)


In [39]:
bert_config.to_dict()

{'vocab_size': 30000,
 'hidden_size': 1024,
 'num_hidden_layers': 24,
 'num_attention_heads': 16,
 'hidden_act': 'gelu',
 'intermediate_size': 4096,
 'hidden_dropout_prob': 0,
 'attention_probs_dropout_prob': 0,
 'max_position_embeddings': 512,
 'type_vocab_size': 2,
 'initializer_range': 0.02,
 'backward_compatible': True,
 'embedding_size': 128,
 'num_hidden_groups': 1,
 'net_structure_type': 0,
 'gap_size': 0,
 'num_memory_blocks': 0,
 'inner_group_num': 1,
 'down_scale_factor': 1}

In [40]:
# we will use this for the Dropout probability between bert layer & final Dense layer
bert_config.hidden_dropout_prob=0.2

In [41]:
# max_seq_length = bert_config.max_position_embeddings
# could even keep max_seq_length as 512, but limiting to 128 for this expt
max_seq_length = 128


#### This is the code which reads data line by line converts & writes the data in tfrecord format

In [42]:
#refer file_based_convert_examples_to_features function in models/official/nlp/data/classifier_data_lib.py
def write_training_data(input_csv_file,output_file,sentencepiece_path=None,sp_proto=None,delimiter='\t',max_seq_length=128):
    ##loading the sentence piece model into memory
    if sp_proto ==None:
        sp_proto = tf.io.gfile.GFile(sentencepiece_path, 'rb').read()
   
    with open(input_csv_file,'r',encoding='utf-8') as csv_file, tf.io.TFRecordWriter(output_file) as tf_record_writer:
        #tried using binary format for bytes_list, but csv_reader requires text format
#     with open(input_csv_file,'rb') as csv_file, tf.io.TFRecordWriter(output_file) as tf_record_writer:
        csv_reader = csv.reader(csv_file,delimiter=delimiter,quotechar='"')
        for i,cols in enumerate(csv_reader):
            ##skipping filename & granular tag (granular tag & final tag are the same here)
#             yield cols[1:-1]
            features = collections.OrderedDict()
            
            uuid = cols[0]
            text = cols[1]
            if(len(cols)<4):
                print('uuid ',uuid)
            intent = cols[2]
            input_example = InputExample(uuid,text,label=intent)
            #this will encode the text,label into ids 
            feature = convert_single_example(ex_index=i,example=input_example,label_list=intent_list,
                                             max_seq_length=max_seq_length,sp_proto=sp_proto)
            features["input_ids"] = _int64_feature(feature.input_ids)
            features["input_mask"] = _int64_feature(feature.input_mask)
            features["segment_ids"] = _int64_feature(feature.segment_ids)
            #Note making it as a list & passing it
            features["label_id"] = _int64_feature([feature.label_id])
#             features["is_real_example"]
            tf_example = tf.train.Example(features=tf.train.Features(feature=features))
            tf_record_writer.write(tf_example.SerializeToString())



In [43]:
write_training_data(train_csv_file,train_data_path,sp_proto=sp_model_proto,max_seq_length=max_seq_length)
write_training_data(eval_csv_file,eval_data_path,sp_proto=sp_model_proto,max_seq_length=max_seq_length)

### Reading the data from tfrecord to provide to training graph
Before we write the actual code to read it let's just explore some of the functions in tf.data api & on how to decode the tfrecord data

In [43]:
raw_ds = tf.data.TFRecordDataset(train_data_path)

In [44]:
def _decode_data(record):
    features = {
        "input_ids":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "input_mask":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "segment_ids":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "label_id":tf.io.FixedLenFeature([],tf.int64)
    }
#     features = {
#         'uuid':tf.io.FixedLenFeature([],tf.string),
#         'text':tf.io.VarLenFeature(tf.int64),
#         'intent':tf.io.FixedLenFeature([],tf.string)
#     }    
    return tf.io.parse_single_example(record,features=features)


In [45]:
## map function processes line by line, similar to spark,scala map or pandas apply fucntion
processed_data = raw_ds.map(_decode_data)

In [46]:
## taking only 2 rows & iterating & viewing output
for line in processed_data.take(2):
    print(line)

{'input_ids': <tf.Tensor: shape=(128,), dtype=int64, numpy=
array([    2,   483,   144,    51,  1071,  1839,    34, 14737,    37,
         236,   818,  1071,    60,     3,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,   

#### Function for reading & creating a dataset object from tfrecord
putting the same code above into a function

In [47]:
def get_classifier_dataset(input_file,seq_length,batch_size,is_training=True):
# this is a simplified version & slightly less optimized version of what is used in official bert training
# refer to function create_classifier_dataset in models/official/nlp/data/create_finetuning_data.py


    # create a tf_data set out of the tfrecords file
    dataset = tf.data.TFRecordDataset(input_file)
    name_to_features = {
        "input_ids":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "input_mask":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "segment_ids":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "label_id":tf.io.FixedLenFeature([],tf.int64)
    }
    ## map function processes line by line, similar to spark,scala map or pandas apply fucntion
    dataset = dataset.map(lambda record: tf.io.parse_single_example(record, name_to_features))
#     or could even do this
#     dataset = dataset.map(_decode_data)
    
#     now separating out the features & label
    def _select_data_from_record(record):
#         x contains the features
#         y is your prediction
# This dataset will be passed to keras's model.fit refer to it's documentation for further details
# a short snippet from that documentation

# A `tf.data` dataset. Should return a tuple
#         of either `(inputs, targets)` or
#         `(inputs, targets, sample_weights)`.

# Even a tuple of values for x works, but list doesn't work
#         x = (
#             record['input_ids'],
#             record['input_mask'],
#             record['segment_ids']
#         )

        x = {
            'input_ids': record['input_ids'],
            'input_mask': record['input_mask'],
            'segment_ids': record['segment_ids']
        }

        y = record['label_id']
        # our dataset is returning a tuple (x,y) - x are the features
        return (x, y)
    
    dataset = dataset.map(_select_data_from_record)
    if is_training:
        dataset = dataset.shuffle(100)
        dataset = dataset.repeat()
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(5)
    return dataset
    

In [48]:
steps_per_loop = 1
learning_rate=1e-5
epochs=10

train_batch_size=16
eval_batch_size=16

train_data_size = input_meta_data['train_data_size']
steps_per_epoch = int(train_data_size / train_batch_size)
print(steps_per_epoch)

warmup_steps = int(epochs * train_data_size * 0.1 / train_batch_size)
eval_steps = int(math.ceil(input_meta_data['eval_data_size'] / eval_batch_size))

1285


In [49]:
training_dataset = get_classifier_dataset(train_data_path,max_seq_length,train_batch_size,is_training=True)
evaluation_dataset = get_classifier_dataset(eval_data_path,max_seq_length,eval_batch_size,is_training=False)

In [50]:
## taking only 2 rows & iterating & viewing output
for line in training_dataset.take(2):
    print(line)

({'input_ids': <tf.Tensor: shape=(16, 128), dtype=int64, numpy=
array([[  2,  13,   1, ...,   0,   0,   0],
       [  2, 420,  21, ...,   0,   0,   0],
       [  2,  13,   1, ...,   0,   0,   0],
       ...,
       [  2, 483, 144, ...,   0,   0,   0],
       [  2,  13,   1, ...,   0,   0,   0],
       [  2,  13,   1, ...,   0,   0,   0]])>, 'input_mask': <tf.Tensor: shape=(16, 128), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'segment_ids': <tf.Tensor: shape=(16, 128), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>}, <tf.Tensor: shape=(16,), dtype=int64, numpy=
array([ 29, 128,  34, 103,  88,  97, 103,  42,  30, 128,  97,  87,  30

## Model code, graph creation
This section is the core logic of the model, here we are using tensorflow hub url for albert model. Using hub simplifies the code a lot      
It's a slightly simplied version of official bert code, that code have functionality to load a non hub model. 
#### refer to models/official/nlp/bert/run_classifier.py
Since in this notebook we are only using one gpu, we can even ignore the strategy part

In [51]:
# strategy = distribution_utils.get_distribution_strategy('mirrored',num_gpus=1)

# strategy = tf.distribute.OneDeviceStrategy("device:GPU:2")
#since devices to use is set to 2 already, only 1 device is visible which is 0
strategy = tf.distribute.OneDeviceStrategy("device:GPU:0")

### Temporary tryouts

In [52]:
bert_keras_layer = hub.KerasLayer(bert_hub_url,trainable=True,tags=None)

In [53]:
bert_loaded = hub.load(bert_hub_url,tags=None)

In [54]:
bert_keras_layer.get_config()

{'name': 'keras_layer',
 'trainable': True,
 'dtype': 'float32',
 'handle': 'https://tfhub.dev/tensorflow/albert_en_large/1'}

In [55]:
temp = bert_loaded.signatures['serving_default']
temp

In [56]:
bert_hub_loaded.sp_model_file.asset_path

<tf.Tensor: shape=(), dtype=string, numpy=b'/space/engineering/tfhub_modules/d0ceaf43f67b8744561ebeeaea4c7c188a6e6f78/assets/30k-clean.model'>

### Defining the graph
It's just this function & your graph definition is done :)

In [57]:
def get_albert_model(bert_config,num_labels,max_seq_length,hub_url):
    """This function return our classification model based on bert + original bert model"""
#     https://www.tensorflow.org/hub/reusable_saved_models
    ## Note that name parameter specified here is the same as the feature name keys in the dictionary (x) in tf.data dataset  
    # Apparantely the code even works even if the name of the input specified here & as the key value to inputs dictionary of tf.keras.Model
    # are different from the tf.data.Dataset input. But let's not rely on that, that seems like a potential bug
    input_ids = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='input_ids')
    input_mask = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='input_mask')
    segment_ids = tf.keras.Input(shape=(max_seq_length),dtype=tf.int32,name='segment_ids')
    
    bert_model = hub.KerasLayer(hub_url,trainable=True,tags=None)
    ### pooled_output will give the representation for [CLS]
    ### sequence_output will give representations for all tokens
    ##since it's classification task we will just use pooled_output
    pooled_output, sequence_output = bert_model([input_ids, input_mask, segment_ids])
    bert_output = tf.keras.layers.Dropout(rate=bert_config.hidden_dropout_prob)(pooled_output) 
    
    initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config.initializer_range)
    output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(bert_output)
    
    return tf.keras.Model(inputs={'input_ids':input_ids,
                                 'input_mask':input_mask,
                                 'segment_ids':segment_ids},
                         outputs=output),bert_model

In [58]:
## clearing the existing graphs
tf.keras.backend.clear_session()

### Let's look at the classification model description/summary
We are calling get_albert_model here just to print out model summary, otherwise we call it internally
in another function

In [59]:
classification_model,core_bert_model = get_albert_model(bert_config=bert_config,
                                                              num_labels=input_meta_data['num_labels'],
                                                              max_seq_length=max_seq_length,
                                                               hub_url=bert_hub_url)

In [60]:
classification_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 17683968    input_ids[0][0]                  
                                                                 input_mask[0][0]             

In [61]:
classification_model.get_config()

{'name': 'model',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 128),
    'dtype': 'int32',
    'sparse': False,
    'ragged': False,
    'name': 'input_ids'},
   'name': 'input_ids',
   'inbound_nodes': []},
  {'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 128),
    'dtype': 'int32',
    'sparse': False,
    'ragged': False,
    'name': 'input_mask'},
   'name': 'input_mask',
   'inbound_nodes': []},
  {'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 128),
    'dtype': 'int32',
    'sparse': False,
    'ragged': False,
    'name': 'segment_ids'},
   'name': 'segment_ids',
   'inbound_nodes': []},
  {'class_name': 'KerasLayer',
   'config': {'name': 'keras_layer',
    'trainable': True,
    'dtype': 'float32',
    'handle': 'https://tfhub.dev/tensorflow/albert_en_large/1'},
   'name': 'keras_layer',
   'inbound_nodes': [[['input_ids', 0, 0, {}],
     ['input_mask', 0, 0, {}],
     ['segment_ids', 0, 0, {

In [62]:
core_bert_model.get_config()

{'name': 'keras_layer',
 'trainable': True,
 'dtype': 'float32',
 'handle': 'https://tfhub.dev/tensorflow/albert_en_large/1'}

#### Define Loss function,  metric

In [63]:
def get_loss_fn(num_classes, loss_factor=1.0):
    """Gets the classification loss function."""

    def classification_loss_fn(labels, logits):
        """Classification loss."""
        labels = tf.squeeze(labels)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        one_hot_labels = tf.one_hot(
            tf.cast(labels, dtype=tf.int32), depth=num_classes, dtype=tf.float32)
        per_example_loss = -tf.reduce_sum(
            tf.cast(one_hot_labels, dtype=tf.float32) * log_probs, axis=-1)
        #batch loss
        loss = tf.reduce_mean(per_example_loss)
        loss *= loss_factor
        return loss
    return classification_loss_fn  


def metric_fn():
    return tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy',dtype=tf.float32)

#### Function for running training

In [64]:
def run_keras_compile_fit(model_dir,
                          strategy,
                          model_fn,
                          training_dataset,
                          evaluation_dataset,
                          loss_fn,
                          metric_fn,
                          init_checkpoint,
                          epochs,
                          steps_per_epoch,
                          steps_per_loop,
                          eval_steps,
                          custom_callbacks=None):
    """Runs BERT classifier model using Keras compile/fit API."""
    ###slightly simplied version of official bert code 
    # refer to models/official/nlp/bert/run_classifier.py -   function run_keras_compile_fit
    # if you running on a single gpu or on just cpu this strategy is not necessary
    with strategy.scope():
        #sub_model is the original bert
        classification_model, sub_model = model_fn()
        optimizer = classification_model.optimizer
        
        # this is not required for the hub version of the model, this restore method is trying to restor values,
        # from a saved bert model (since we have only provided the sub_model to checkpoint)
        # Let's saying we stopped training at some point & want to continue from that point next time,
        # here we can instead load our classification_model & init_checkpoint can be our previous saved checkpoint file
        if init_checkpoint:
            checkpoint = tf.train.Checkpoint(model=sub_model)
            checkpoint.restore(init_checkpoint).assert_existing_objects_matched()

        classification_model.compile(
            optimizer=optimizer,
            loss=loss_fn,
            metrics=[metric_fn()])
    #     ,experimental_steps_per_execution=steps_per_loop)

        summary_dir = os.path.join(model_dir, 'summaries')
        summary_callback = tf.keras.callbacks.TensorBoard(summary_dir)
        checkpoint_path = os.path.join(model_dir, 'checkpoint')
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            checkpoint_path, save_weights_only=True)
        early_stopping = tf.keras.callbacks.EarlyStopping()

        if custom_callbacks is not None:
            custom_callbacks += [summary_callback, checkpoint_callback, early_stopping]
        else:
            custom_callbacks = [summary_callback, checkpoint_callback, early_stopping]
#       Note that we are only passing x & not y in fit function
#       Refer to keras model.fit documentation for further details

#       If `x` is a dataset, generator,
#       or `keras.utils.Sequence` instance, `y` should
#       not be specified (since targets will be obtained from `x`).
        model_history = classification_model.fit(
            x=training_dataset,
            validation_data=evaluation_dataset,
            steps_per_epoch=steps_per_epoch,
            epochs=epochs,
            validation_steps=eval_steps,
            callbacks=custom_callbacks)

        return classification_model

In [65]:
output_folder

'/var/extra/users/jgeorge/tf2.0/input/dish/models/albert_en_large'

### The training function call

In [66]:
## clearing the existing graphs
tf.keras.backend.clear_session()

In [67]:
loss_multiplier = 1
max_seq_length = input_meta_data['max_seq_length']
num_classes = input_meta_data['num_labels']
loss_fn = get_loss_fn(num_classes,loss_multiplier)
epochs=20
initial_lr = learning_rate
def _get_classifier_model():
    classifier_model,core_model = get_albert_model(bert_config=bert_config,
                                                              num_labels=num_classes,
                                                              max_seq_length=max_seq_length,
                                                               hub_url=bert_hub_url)
    ##This is basically Adam optimizer with weight decay after set no of warm up steps & 
    # before that an increasing learning rate from 0 to initial_lr
    # refer to models/official/nlp/optimization.py for more details
    classifier_model.optimizer = optimization.create_optimizer(init_lr=initial_lr,
                                                               num_train_steps=steps_per_epoch*epochs,
                                                               num_warmup_steps=warmup_steps)
    return classifier_model,core_model

trained_model = run_keras_compile_fit(model_dir=output_folder,strategy=strategy,model_fn=_get_classifier_model,
                                     training_dataset=training_dataset,evaluation_dataset=evaluation_dataset,
                                      loss_fn=loss_fn,metric_fn=metric_fn,
                                     init_checkpoint=None,
                                      epochs=epochs,
                                      steps_per_epoch=steps_per_epoch,
                                      steps_per_loop=steps_per_loop,
                                      eval_steps=eval_steps
                                     )

Epoch 1/20
1285/1285 [==============================] - 967s 752ms/step - loss: 4.1694 - test_accuracy: 0.1472 - val_loss: 2.9912 - val_test_accuracy: 0.4090
Epoch 2/20
1285/1285 [==============================] - 969s 754ms/step - loss: 2.3306 - test_accuracy: 0.5118 - val_loss: 1.8371 - val_test_accuracy: 0.6059
Epoch 3/20
1285/1285 [==============================] - 970s 755ms/step - loss: 1.5524 - test_accuracy: 0.6758 - val_loss: 1.4411 - val_test_accuracy: 0.6897
Epoch 4/20
1285/1285 [==============================] - 971s 756ms/step - loss: 1.1426 - test_accuracy: 0.7578 - val_loss: 1.2217 - val_test_accuracy: 0.7329
Epoch 5/20
1285/1285 [==============================] - 972s 757ms/step - loss: 0.8821 - test_accuracy: 0.8107 - val_loss: 1.1210 - val_test_accuracy: 0.7496
Epoch 6/20
1285/1285 [==============================] - 973s 757ms/step - loss: 0.7038 - test_accuracy: 0.8472 - val_loss: 1.0250 - val_test_accuracy: 0.7747
Epoch 7/20
1285/1285 [==============================

## Saving the model
The model that we trained right now takes the sentence piece encoded list as input, along with input_mask list, segment_ids list
When we are deploying we want to give a text as input & get the text label as output.   
So what we are going to do is to have a graph with sentence piece encoding within the graph itself & create input_mask, segment_ids also as part of the graph for input   
For the output have a softmax & a table lookup to on labels to return the label name 

### Sentence piece tokenizaton
I'll walk through details of this Layer after we are done saving the model

In [ ]:
tf.keras.layers.Layer()

In [3]:
import tensorflow as tf

In [68]:
# @tf.keras.utils.register_keras_serializable(package="Text")

class SentencepieceTokenization(tf.keras.layers.Layer):
    def __init__(self,model_path,max_seq_length):
        super(SentencepieceTokenization, self).__init__(trainable=False,dtype=tf.int32)
        self.model_path = model_path
        # TODO keep a tf.constant else tracing issues 
        self.max_seq_length = max_seq_length
        
    def build(self,input_shape):
        self.model_proto = tf.io.gfile.GFile(self.model_path, 'rb').read()
        ## pad_id is usually 0 
        [self.CLS_ID,self.SEP_ID,self.PAD_ID]  = tfs.piece_to_id(['[CLS]','[SEP]','<pad>'],model_file=self.model_path)
        self.built=True
        
#     @tf.function    
    def call(self, input_text):
        ##tensorflow sentence piece works while exporting to graph while, tf_text sentencepiece doesn't
        encoded_text = self.get_encoded_text(input_text,max_sequence_length=self.max_seq_length)
        return encoded_text

#     @tf.function
    def get_encoded_text(self,input_text_batch,max_sequence_length):
        def process_invidual_line_encoding(x):
#             tf sentencepiece requires a list as input, while the individual value that we get here
#             is a single sentence, so adding one more dimension (i.e adding batch dimension = 1)
            list_x = tf.expand_dims(x,axis=0)
            sp_encoded = tfs.encode(list_x,model_proto=self.model_proto)
    #         removing the batch dim with size=1 (which we added in the previous step)
            values = tf.squeeze(sp_encoded.values,name='squeezed_values')
            sequence_length = tf.squeeze(sp_encoded.sequence_length)
    #         trimming to max_length-2 (-2 to incorporate [CLS], [SEP])
            trimmed_max_length = max_sequence_length-2
            values_trimmed = tf.cond(tf.greater(sequence_length,trimmed_max_length), 
                                    lambda : tf.slice(values,begin=[0],size=[trimmed_max_length],name='trimmed_out'),lambda : values)
            concat = tf.concat([[self.CLS_ID],values_trimmed,[self.SEP_ID]],axis=0,name='concat_out')
            # tf.shape gives dynamic shape &
            # tenorflow_variable.shape gives static shape with dynamic entries = None
            actual_token_length = tf.shape(concat)[-1]
            # need not prepend anything so 0 for 1st entry in padding, 
            # & next value for padding is how many dimensions required at the end of tensor
            padded = tf.pad(concat,paddings=[[0,max_sequence_length-actual_token_length]],name='input_ids')
    #         segment_ids = tf.zeros(shape=tf.shape(padded),dtype=tf.int32)
    #         or
            # This will create zeros with similar shape as padded tensor
            segment_ids = tf.zeros_like(padded,dtype=tf.int32,name='segment_ids')
            
            # This tensor is zero initialized & will update 0th index till actual_token_length with 1
            input_mask = tf.scatter_nd(indices=tf.expand_dims(tf.range(0,actual_token_length),axis=1),
                                       updates=tf.ones(shape=[actual_token_length],dtype=tf.int32),
                                       shape=[max_sequence_length],name='input_mask')
            return (padded,segment_ids,input_mask)
    #     Issue running map_fn on gpu https://github.com/tensorflow/tensorflow/issues/28007 
    #     https://www.tensorflow.org/api_docs/python/tf/device
        with tf.device('/device:CPU:0'):
            encoded = tf.map_fn(lambda x: process_invidual_line_encoding(x),input_text_batch,dtype=(tf.int32,tf.int32,tf.int32))
            return encoded
    
    
    def get_config(self):
        config = super(SentencepieceTokenization, self).get_config()
        config.update({'model_path': self.model_path})
        return config


In [69]:
class LabelLookup(tf.keras.layers.Layer):
#     def __init__(self,model_proto):
    def __init__(self,filepath,default):
        """ 
        filepath - path to the text file with 1 label per line
        """
        super(LabelLookup, self).__init__(trainable=False,dtype=tf.int32)
        self.filepath = filepath
        self.default=default
        
    def build(self,input_shape):
        # need to convert label index to label name. So keeping line no as key & whole text line as label
        # issue with using table lookup (both file based & key value tensorbased) in tf2.2 (tf2.x) - https://github.com/tensorflow/tensorflow/issues/38305
        self.table = tf.lookup.StaticHashTable(
            tf.lookup.TextFileInitializer(self.filepath,
                                          key_dtype=tf.int64,key_index=tf.lookup.TextFileIndex.LINE_NUMBER,
                                     value_dtype=tf.string,value_index=tf.lookup.TextFileIndex.WHOLE_LINE),self.default)
        self.built=True
        
    def call(self, input_text):
        word_ids = self.table.lookup(input_text)
        return word_ids
    
    def get_config(self):
        config = super(LabelLookup, self).get_config()
#         config.update({'filepath': self.filepath})
        return config


In [70]:
max_seq_length = input_meta_data['max_seq_length']
num_labels = input_meta_data['num_labels']
initial_lr = learning_rate
default_label_index = 1
default_label='other-other'
checkpoint_path=tf.train.latest_checkpoint(output_folder)

input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')
sentence_piece_layer = SentencepieceTokenization(model_path=sentencepiece_path,max_seq_length=tf.constant(max_seq_length))
input_ids_processed,segment_ids_processed,input_mask_processed = sentence_piece_layer(input_text)


bert_model = hub.KerasLayer(bert_hub_url,trainable=True,tags=None)
### pooled_output will give the representation for [CLS]
### sequence_output will give representations for all tokens
## since it's classification task we will just use pooled_output
pooled_output, sequence_output = bert_model([input_ids_processed, input_mask_processed, segment_ids_processed])
#     bert_output = tf.keras.layers.Dropout(rate=bert_config.hidden_dropout_prob)(pooled_output) 

# This doesn't really matter as we will be loading the weights from saved checkpoint
initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config.initializer_range)
#     output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(bert_output)
# dense_output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32,activation='softmax')(pooled_output)
dense_output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(pooled_output)
# log_probs = tf.nn.softmax()
log_probs = tf.nn.log_softmax(dense_output, axis=-1)
label_id = tf.argmax(log_probs,axis=-1)

label = LabelLookup(filepath=intent_file,default=default_label)(label_id)

loaded_model  = tf.keras.Model(inputs={'input_text':input_text},
                     outputs=label)
# loaded_model.compile()
#     checkpoint = tf.train.Checkpoint(model=model)
#     checkpoint.restore(checkpoint_path).assert_existing_objects_matched()
loaded_model.load_weights(checkpoint_path).assert_existing_objects_matched()


Two checkpoint references resolved to different objects (<tensorflow_hub.keras_layer.KerasLayer object at 0x7fad24620dd0> and <tensorflow.python.keras.layers.core.Dense object at 0x7facac64c3d0>).



Two checkpoint references resolved to different objects (<tensorflow_hub.keras_layer.KerasLayer object at 0x7fad24620dd0> and <tensorflow.python.keras.layers.core.Dense object at 0x7facac64c3d0>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7facac64c3d0> and <tensorflow.python.keras.engine.base_layer.TensorFlowOpLayer object at 0x7fabd81a50d0>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.core.Dense object at 0x7facac64c3d0> and <tensorflow.python.keras.engine.base_layer.TensorFlowOpLayer object at 0x7fabd81a50d0>).


In [71]:
loaded_model.predict(['talk to agent'])

array([b'representative-request'], dtype=object)

In [93]:
input_ids_processed

<tf.Tensor 'sentencepiece_tokenization_2/Identity:0' shape=(None, None) dtype=int32>

In [76]:
label_map = {i:label for i,label in enumerate(intent_list)}

In [77]:
label_map.get(131)

'representative-request'

#### Let's put the loading model part into a function

In [94]:
def get_albert_model_saving(bert_config,input_meta_data,hub_url,checkpoint_path,sentencepiece_path,label_file):
    """This function return our classification model based on bert + updated bert part of model """

    max_seq_length = input_meta_data['max_seq_length']
    num_labels = input_meta_data['num_labels']
    default_label = tf.constant(input_meta_data['default_label'])
    #alternative way of giving input by specifying the entire text & do the preprocessing in the graph
    input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')
    sentence_piece_layer = SentencepieceTokenization(model_path=sentencepiece_path,max_seq_length=tf.constant(max_seq_length))
    input_ids_processed,segment_ids_processed,input_mask_processed = sentence_piece_layer(input_text)
    
    
    bert_model = hub.KerasLayer(hub_url,trainable=True,tags=None)
    ### pooled_output will give the representation for [CLS]
    ### sequence_output will give representations for all tokens
    ##since it's classification task we will just use pooled_output
    pooled_output, sequence_output = bert_model([input_ids_processed, input_mask_processed, segment_ids_processed])
    
    # This doesn't really matter as we will be loading the weights from saved checkpoint
    initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config.initializer_range)
    # we don't want dropout for the model we are going to serve so skipping that
    dense_output = tf.keras.layers.Dense(num_labels,kernel_initializer=initializer,name='output',dtype=tf.float32)(pooled_output)
    # log_probs = tf.nn.softmax()
    log_probs = tf.nn.log_softmax(dense_output, axis=-1)
    label_id = tf.argmax(log_probs,axis=-1)

    label = LabelLookup(filepath=label_file,default=default_label)(label_id)

    model  = tf.keras.Model(inputs={'input_text':input_text},
                         outputs={'label':label,'score':log_probs[label_id]})
    
#     checkpoint = tf.train.Checkpoint(model=model)
#     checkpoint.restore(checkpoint_path).assert_existing_objects_matched()
    model.load_weights(checkpoint_path).assert_existing_objects_matched()
    return model,bert_model, input_text,label,log_probs

In [92]:
temp = tf.constant([1.0,2.0])
temp

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>

In [93]:
temp[0]

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [98]:
tf.slice(temp,[1],[1])

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.], dtype=float32)>

In [ ]:
tfs.piece_to_id()

In [78]:
print(sentencepiece_path)
sentencepiece_path = sentencepiece_path.decode('utf-8')
print(sentencepiece_path)

b'/space/engineering/tfhub_modules/d0ceaf43f67b8744561ebeeaea4c7c188a6e6f78/assets/30k-clean.model'
/space/engineering/tfhub_modules/d0ceaf43f67b8744561ebeeaea4c7c188a6e6f78/assets/30k-clean.model


In [95]:
tf.keras.backend.clear_session()

In [75]:
label_file = intent_file

In [81]:
output_folder

'/var/extra/users/jgeorge/tf2.0/input/dish/models/albert_en_large'

In [96]:
classification_model_to_save, bert_updated,input_text,label,log_probs = get_albert_model_saving(bert_config,
                                                                     input_meta_data,
                                                                     bert_hub_url,
                                                                     checkpoint_path=tf.train.latest_checkpoint(output_folder),
                                                                    sentencepiece_path=sentencepiece_path,
                                                                     label_file=label_file
                                                                    )

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got <tf.Tensor 'ArgMax:0' shape=(None,) dtype=int64>

In [84]:
# final_model_path=os.path.join(output_folder,'final_model')
final_model_path3='/space/engineering/tf_serve/models/tryout_jithin_3'
classification_model_to_save.save(os.path.join(final_model_path3,'1'))

INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/tryout_jithin_3/1/assets


INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/tryout_jithin_3/1/assets


In [90]:
classification_model_to_save.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_text (InputLayer)         [(None,)]            0                                            
__________________________________________________________________________________________________
sentencepiece_tokenization (Sen ((None, None), (None 0           input_text[0][0]                 
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 17683968    sentencepiece_tokenization[0][0] 
                                                                 sentencepiece_tokenization[0][2] 
                                                                 sentencepiece_tokenization[0][1] 
______________________________________________________________________________________________

In [86]:
input_meta_data

{'max_seq_length': 128,
 'num_labels': 142,
 'train_data_size': 20574,
 'eval_data_size': 5144,
 'default_label': 'other-other'}

In [91]:
class CustomModel(tf.keras.Model):
    
    def __init__(self,input_meta_data,hub_url):
        super(CustomModel, self).__init__()
        self.max_seq_length = input_meta_data['max_seq_length']
        self.num_labels = input_meta_data['num_labels']
        self.default_label = tf.constant(input_meta_data['default_label'])
        self.sentence_piece_layer = SentencepieceTokenization(model_path=sentencepiece_path,max_seq_length=tf.constant(max_seq_length))
        self.bert_model = hub.KerasLayer(hub_url,trainable=True,tags=None)
        # This doesn't really matter as we will be loading the weights from saved checkpoint
        self.initializer = tf.keras.initializers.TruncatedNormal(stddev=bert_config.initializer_range)
        # we don't want dropout for the model we are going to serve so skipping that
        self.dense_final = tf.keras.layers.Dense(num_labels,kernel_initializer=self.initializer,name='output',dtype=tf.float32)
        self.label_lookup = LabelLookup(filepath=label_file,default=default_label)
        
    @tf.function(input_signature=[tf.TensorSpec(shape=None,dtype=tf.string)])
    def call(input_text):
        return get_classified_output(input_text)
    
    @tf.function(input_signature=[tf.TensorSpec(shape=None,dtype=tf.string)])
    def get_bert_embedding(input_text):
        input_ids_processed,segment_ids_processed,input_mask_processed = self.sentence_piece_layer(input_text)
        
        ### pooled_output will give the representation for [CLS]
        ### sequence_output will give representations for all tokens
        ##since it's classification task we will just use pooled_output
        pooled_output, sequence_output = self.bert_model([input_ids_processed, input_mask_processed, segment_ids_processed])

        return {'pooled_output':pooled_output}

    @tf.function(input_signature=[tf.TensorSpec(shape=None,dtype=tf.string)])
    def get_classified_output(input_text):
        pooled_dict = get_bert_embedding(input_text)
        pooled_output = pooled_dict['pooled_output']

        # log_probs = tf.nn.softmax()
        dense_output= self.dense_final(pooled_output)
        log_probs = tf.nn.log_softmax(dense_output, axis=-1)
        label_id = tf.argmax(log_probs,axis=-1)

        label = self.label_lookup(label_id)

    #     model  = tf.keras.Model(inputs={'input_text':input_text},
    #                          outputs=label)

    #     checkpoint = tf.train.Checkpoint(model=model)
    #     checkpoint.restore(checkpoint_path).assert_existing_objects_matched()
    #     model.load_weights(checkpoint_path).assert_existing_objects_matched()
        return label
tf.keras.backend.clear_session()
input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')
custom_model=CustomModel(input_meta_data,bert_hub_url)
checkpoint_path=tf.train.latest_checkpoint(output_folder)
custom_model.load_weights(checkpoint_path).assert_existing_objects_matched()

AssertionError: Nothing except the root object matched a checkpointed value. Typically this means that the checkpoint does not match the Python program. The following objects have no matching checkpointed value: [<tf.Variable 'position_embedding/embeddings:0' shape=(512, 128) dtype=float32, numpy=
array([[ 0.04171413, -0.05951454,  0.00528023, ..., -0.00547502,
        -0.00608543,  0.01891393],
       [ 0.01842086, -0.00854636,  0.01457946, ..., -0.2188653 ,
        -0.02077591,  0.05037078],
       [-0.02886076, -0.01502202,  0.02091144, ..., -0.16695945,
        -0.0303349 ,  0.06738311],
       ...,
       [ 0.05315233,  0.00108237,  0.01054862, ..., -0.11003321,
         0.00282854, -0.01345602],
       [ 0.02562682, -0.06601355, -0.01717755, ..., -0.13026832,
        -0.00394623,  0.04304561],
       [ 0.02484268, -0.03633654, -0.03465453, ..., -0.08623783,
        -0.00122317,  0.00660813]], dtype=float32)>, <tf.Variable 'transformer/self_attention/query/bias:0' shape=(16, 64) d

In [ ]:
tf.keras.Model()

In [ ]:
load_final_model_keras = tf.keras.models.load_model(final_model_path)

In [91]:
loaded_final_model = tf.saved_model.load(final_model_path)

In [93]:
serving_default = loaded_final_model.signatures['serving_default']
serving_default

In [94]:
serving_default.structured_input_signature

((),
 {'input_text': TensorSpec(shape=(None,), dtype=tf.string, name='input_text')})

In [95]:
serving_default.structured_outputs

{'label_lookup': TensorSpec(shape=(None,), dtype=tf.string, name='label_lookup')}

In [96]:
serving_default(tf.constant(['talk to an agent']))

{'label_lookup': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'other-other'], dtype=object)>}

In [ ]:
classification_model_to_save, bert_updated,input_text,label,log_probs

In [104]:
tf.compat.v1.saved_model.predict_signature_def(inputs={'input_text':input_text},
                                                           outputs={'output_label':label})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


RuntimeError: build_tensor_info is not supported in Eager mode.

In [105]:
classification_model_to_save.

In [ ]:
@tf.function(input_signature=[tf.TensorSpec(shape=(None,), dtype=tf.string)])
  def add(self, ):
    return x + x + 1.


In [ ]:
@tf.function(input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)])
  def add(self, x):
    return x + x + 1.


In [99]:
tf.saved_model.save()

'serve'

In [ ]:
tf.saved_model.save()

In [ ]:
final_model_with_signature_path=os.path.join(output_folder,'final_model_with_signature')
classification_model_to_save.save(final_model_with_signature_path,signatures=)

In [ ]:
loaded_final_model.signatures['serving_default'](['talk to an agent'])

In [159]:
loaded_final_model(['talk to agent','talk to agent','talk to agent','talk to agent'])

ValueError: Could not find matching function to call loaded from the SavedModel. Got:
  Positional arguments (3 total):
    * ['talk to agent', 'talk to agent', 'talk to agent', 'talk to agent']
    * False
    * None
  Keyword arguments: {}

Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (3 total):
    * {'input_text': TensorSpec(shape=(None,), dtype=tf.string, name='input_text')}
    * True
    * None
  Keyword arguments: {}

Option 2:
  Positional arguments (3 total):
    * {'input_text': TensorSpec(shape=(None,), dtype=tf.string, name='inputs/input_text')}
    * False
    * None
  Keyword arguments: {}

Option 3:
  Positional arguments (3 total):
    * {'input_text': TensorSpec(shape=(None,), dtype=tf.string, name='input_text')}
    * False
    * None
  Keyword arguments: {}

Option 4:
  Positional arguments (3 total):
    * {'input_text': TensorSpec(shape=(None,), dtype=tf.string, name='inputs/input_text')}
    * True
    * No

In [120]:
tf.lookup.StaticHashTable.mro()

[tensorflow.python.ops.lookup_ops.StaticHashTable,
 tensorflow.python.ops.lookup_ops.InitializableLookupTableBase,
 tensorflow.python.ops.lookup_ops.LookupInterface,
 tensorflow.python.training.tracking.tracking.TrackableResource,
 tensorflow.python.training.tracking.tracking.CapturableResource,
 tensorflow.python.training.tracking.base.Trackable,
 object]

In [62]:
%load_ext tensorboard

In [ ]:
!ls /var/extra/users/jgeorge/tf2.0/input/dish/models/albert_en_large/checkpoint

In [219]:
def get_input_predict(row):
    text  = row['text']
    d = {}
    t = []
    t.append(2)
    temp= tfs.encode([text],model_proto=sp_model).values.numpy()[0]
    t.extend(temp)
    t.append(3)
    
#     d['input_ids'] = [t]
#     d['input_mask'] = [[1]*len(t)]
#     d['segment_ids'] = [[0]*len(t)]
    
    d['input_ids'] = tf.convert_to_tensor([t],dtype=tf.int32)
    d['input_mask'] = tf.convert_to_tensor([[1]*len(t)],dtype=tf.int32)
    d['segment_ids'] = tf.convert_to_tensor([[0]*len(t)],dtype=tf.int32)
#     return pd.Series(d)
    return d


In [220]:
temp_test = df.head()
temp_test = temp_test.apply(lambda x: get_input_predict(x),axis=1)
temp_test

0    {'input_ids': ((tf.Tensor(2, shape=(), dtype=i...
1    {'input_ids': ((tf.Tensor(2, shape=(), dtype=i...
2    {'input_ids': ((tf.Tensor(2, shape=(), dtype=i...
3    {'input_ids': ((tf.Tensor(2, shape=(), dtype=i...
4    {'input_ids': ((tf.Tensor(2, shape=(), dtype=i...
dtype: object

In [222]:
temp_test[0]

{'input_ids': <tf.Tensor: shape=(1, 11), dtype=int32, numpy=
 array([[   2,   92,   42, 2660,   51, 1071,   20,   51, 4216,   60,    3]],
       dtype=int32)>,
 'input_mask': <tf.Tensor: shape=(1, 11), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>,
 'segment_ids': <tf.Tensor: shape=(1, 11), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}

In [221]:
trained_model.predict(temp_test)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type dict).

In [201]:
temp_test['input_ids'].values

array([list([2, 92, 42, 2660, 51, 1071, 20, 51, 4216, 60, 3]),
       list([2, 13, 1, 93, 13, 1, 1326, 6264, 63, 529, 13, 1, 58, 4213, 2119, 2800, 19, 51, 634, 337, 9, 13, 1, 438, 3320, 6264, 63, 2800, 9, 3]),
       list([2, 13, 1, 376, 21, 7582, 3896, 86, 31, 221, 22, 38, 164, 51, 365, 2830, 576, 1427, 3]),
       list([2, 184, 92, 31, 477, 51, 375, 6328, 9, 32, 2206, 31, 1049, 57, 186, 3]),
       list([2, 13, 1, 1830, 69, 20, 1960, 29, 737, 88, 51, 1071, 3])],
      dtype=object)

In [202]:
predict_input = [temp_test['input_ids'].values,temp_test['input_mask'].values,temp_test['segment_ids'].values]

In [211]:
temp_test.to_dict(orient='list')

{'input_ids': [[2, 92, 42, 2660, 51, 1071, 20, 51, 4216, 60, 3],
  [2,
   13,
   1,
   93,
   13,
   1,
   1326,
   6264,
   63,
   529,
   13,
   1,
   58,
   4213,
   2119,
   2800,
   19,
   51,
   634,
   337,
   9,
   13,
   1,
   438,
   3320,
   6264,
   63,
   2800,
   9,
   3],
  [2,
   13,
   1,
   376,
   21,
   7582,
   3896,
   86,
   31,
   221,
   22,
   38,
   164,
   51,
   365,
   2830,
   576,
   1427,
   3],
  [2, 184, 92, 31, 477, 51, 375, 6328, 9, 32, 2206, 31, 1049, 57, 186, 3],
  [2, 13, 1, 1830, 69, 20, 1960, 29, 737, 88, 51, 1071, 3]],
 'input_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'segment_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0

In [206]:

def get_predict_dataset(input_file,seq_length,batch_size,is_training=True):
# this is a simplified version & slightly less optimized version of what is used in official bert training
# refer to function create_classifier_dataset in models/official/nlp/data/create_finetuning_data.py


    # create a tf_data set out of the tfrecords file
    dataset = tf.data.TFRecordDataset(input_file)
    name_to_features = {
        "input_ids":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "input_mask":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "segment_ids":tf.io.FixedLenFeature([max_seq_length],tf.int64),
        "label_id":tf.io.FixedLenFeature([],tf.int64)
        }
    ## map function processes line by line, similar to spark,scala map or pandas apply fucntion
    dataset = dataset.map(lambda record: tf.io.parse_single_example(record, name_to_features))
#     now separating out the features & label
    def _select_data_from_record(record):
#         x contains the features
#         y is your prediction
#This dataset will be passed to keras's model.fit refer to it's documentation for further details
# a short snippet from that documentation

# A `tf.data` dataset. Should return a tuple
#         of either `(inputs, targets)` or
#         `(inputs, targets, sample_weights)`.


        x = {
            'input_ids': record['input_ids'],
            'input_mask': record['input_mask'],
            'segment_ids': record['segment_ids']
        }
        y = record['label_id']
        return x
    
    dataset = dataset.map(_select_data_from_record)
    if is_training:
        dataset = dataset.shuffle(100)
        dataset = dataset.repeat()
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(1024)
    return dataset
prediction_dataset = get_predict_dataset(eval_data_path,max_seq_length,eval_batch_size,is_training=False)    

In [224]:
for line in prediction_dataset.take(5):
    print(line)

{'input_ids': <tf.Tensor: shape=(16, 128), dtype=int64, numpy=
array([[   2,  184,  107, ...,    0,    0,    0],
       [   2,   13,    1, ...,    0,    0,    0],
       [   2,  184,  107, ...,    0,    0,    0],
       ...,
       [   2, 5388, 2697, ...,    0,    0,    0],
       [   2,   13,    1, ...,    0,    0,    0],
       [   2,   31,   41, ...,    0,    0,    0]])>, 'input_mask': <tf.Tensor: shape=(16, 128), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'segment_ids': <tf.Tensor: shape=(16, 128), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>}
{'input_ids': <tf.Tensor: shape=(16, 128), dtype=int64, numpy=
array([[ 2, 92, 13

In [207]:
trained_model.predict(prediction_dataset)

array([[ 0.9895891 ,  0.38930255, -0.2897632 , ...,  1.347748  ,
        -0.7115326 ,  0.43504658],
       [-0.5799464 , -0.69323003,  0.49325743, ...,  1.1278716 ,
        -0.97258717, -0.09923831],
       [-0.9445586 , -1.2251675 , -0.37769505, ..., -0.8239982 ,
        -0.5968072 , -0.9161253 ],
       ...,
       [-1.640143  , -0.58176893, -1.2308036 , ..., -0.83895874,
        -0.7102679 , -1.271832  ],
       [ 0.96426886, -0.1527807 , -0.3959958 , ...,  2.149941  ,
        -0.50963145,  0.30589893],
       [ 1.1327688 ,  1.637278  , -1.304856  , ...,  3.4651942 ,
        -0.15032795,  0.1539227 ]], dtype=float32)

In [ ]:
classifier_model,core_model = get_albert_model(bert_config=bert_config,
                                                               float_type=tf.float32,
                                                              num_labels=num_classes,
                                                              max_seq_length=max_seq_length,
                                                               hub_module_url=bert_hub_url)

In [104]:
def get_dataset_fn(input_file_pattern, max_seq_length, global_batch_size,
                   is_training):
  """Gets a closure to create a dataset."""
  def _dataset_fn(ctx=None):
    """Returns tf.data.Dataset for distributed BERT pretraining."""
    batch_size = ctx.get_per_replica_batch_size(
        global_batch_size) if ctx else global_batch_size
#     dataset = input_pipeline.create_classifier_dataset(
    dataset = create_classifier_dataset(
        input_file_pattern,
        max_seq_length,
        batch_size,
        is_training=is_training,
        input_pipeline_context=ctx)
    return dataset
  return _dataset_fn


In [105]:
train_input_fn = get_dataset_fn(
#       FLAGS.train_data_path,
      train_data_path,
      max_seq_length,
#       FLAGS.train_batch_size,
      train_batch_size,
      is_training=True)
eval_input_fn = get_dataset_fn(
#       FLAGS.eval_data_path,
      eval_data_path,
      max_seq_length,
#       FLAGS.eval_batch_size,
      eval_batch_size,
      is_training=False)


In [119]:
albert_hub_url

'https://tfhub.dev/google/albert_base/2'

In [148]:
output_folder

'/var/extra/users/jgeorge/tf2.0/input/dish/models/albert_en_large'

In [172]:
# init_checkpoint=output_folder
init_checkpoint=None

In [161]:
bert_model = hub.KerasLayer(albert_hub_url,trainable=True,tags=None)

In [169]:
bert_config=albert_config

In [174]:
classifier_model,core_model = combine_model(bert_config=albert_config,
                                                               float_type=tf.float32,
                                                              num_labels=num_classes,
                                                              max_seq_length=max_seq_length,
                                                               hub_module_url=albert_hub_url)

 bert_model  <tensorflow_hub.keras_layer.KerasLayer object at 0x7f70b5d017d0>
pooled_output  Tensor("keras_layer_15/Identity:0", shape=(None, 1024), dtype=float32)
bert_output Tensor("dropout_4/Identity:0", shape=(None, 1024), dtype=float32)


In [ ]:
classifier_model.compile()

In [135]:
trained_model = run_bert_classifier(
      strategy,
      albert_config,
      input_meta_data,
      albert_model_dir,
      epochs,
      steps_per_epoch,
      steps_per_loop,
      eval_steps,
      warmup_steps,
      learning_rate,
      init_checkpoint,
      train_input_fn,
      eval_input_fn,
      run_eagerly=False,
      use_keras_compile_fit=False)


https://tfhub.dev/tensorflow/albert_en_large/1


ValueError: The SavedModel at /tmp/tfhub_modules/d0ceaf43f67b8744561ebeeaea4c7c188a6e6f78 has one MetaGraph with tags ['serve'], but got an incompatible argument tags={'train'} to tf.saved_model.load. You may omit it, pass 'None', or pass matching tags.

In [85]:
#clear the existing tensorflow graph
tf.keras.backend.clear_session()

In [114]:
def run_bert_classifier(strategy,
                        bert_config,
                        input_meta_data,
                        model_dir,
                        epochs,
                        steps_per_epoch,
                        steps_per_loop,
                        eval_steps,
                        warmup_steps,
                        initial_lr,
                        init_checkpoint,
                        train_input_fn,
                        eval_input_fn,
                        custom_callbacks=None,
                        run_eagerly=False,
                        use_keras_compile_fit=False):
    """Run BERT classifier training using low-level API."""
    max_seq_length = input_meta_data['max_seq_length']
    num_classes = input_meta_data['num_labels']
    def _get_classifier_model():
#     bert_models.classifier_model
#     classifier_model,core_model = get_classifier_model(bert_config=bert_config,
#                                                                float_type=tf.float32,
#                                                               num_labels=num_classes,
#                                                               max_seq_length=max_seq_length,
#                                                                hub_module_url=albert_hub_url)
        classifier_model,core_model = combine_model(bert_config=bert_config,
                                                                   float_type=tf.float32,
                                                                  num_labels=num_classes,
                                                                  max_seq_length=max_seq_length,
                                                                   hub_module_url=albert_hub_url)
        classifier_model.optimizer = optimization.create_optimizer(init_lr=initial_lr,
                                                                   num_train_steps=steps_per_epoch*epochs,
                                                                   num_warmup_steps=warmup_steps)
        return classifier_model,core_model
    loss_multiplier = 1
    loss_fn = get_loss_fn(num_classes,loss_multiplier)
    
    def metric_fn():
        return tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy',dtype=tf.float32)
    return model_training_utils.run_customized_training_loop(
      strategy=strategy,
      model_fn=_get_classifier_model,
      loss_fn=loss_fn,
      model_dir=model_dir,
      steps_per_epoch=steps_per_epoch,
      steps_per_loop=steps_per_loop,
      epochs=epochs,
      train_input_fn=train_input_fn,
      eval_input_fn=eval_input_fn,
      eval_steps=eval_steps,
      init_checkpoint=init_checkpoint,
      metric_fn=metric_fn,
      custom_callbacks=custom_callbacks,
      run_eagerly=run_eagerly)


In [87]:
albert_model_dir

'/var/extra/users/jgeorge/tf2.0/input/albert_base'

In [ ]:
strategy.experimental_run_v2()

In [94]:
model_dir_dish=os.path.join(main_exp_folder,'models')


In [97]:
model_dir_dish

'/var/extra/users/jgeorge/tf2.0/input/dish/data/jan17_2020/models'

In [98]:
init_checkpoint=None

In [107]:
albert_config

In [115]:
trained_model = run_bert_classifier(
      strategy,
      albert_config,
      input_meta_data,
      albert_model_dir,
      epochs,
      steps_per_epoch,
      steps_per_loop,
      eval_steps,
      warmup_steps,
      learning_rate,
      init_checkpoint,
      train_input_fn,
      eval_input_fn,
      run_eagerly=False,
      use_keras_compile_fit=False)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


 bert_model  <tensorflow_hub.keras_layer.KerasLayer object at 0x7f7402ebb150>


ValueError: in user code:

    /opt/custom/python/anaconda3/envs/tf2.2/lib/python3.7/site-packages/tensorflow_hub/keras_layer.py:206 call  *
        self._check_trainability()
    /opt/custom/python/anaconda3/envs/tf2.2/lib/python3.7/site-packages/tensorflow_hub/keras_layer.py:265 _check_trainability  *
        raise ValueError(

    ValueError: Setting hub.KerasLayer.trainable = True is unsupported when loading from the hub.Module format of TensorFlow 1.


In [ ]:
trained_model.save()

In [60]:
trained_model.evaluate(eval_input_fn)

ValueError: Failed to find data adapter that can handle input: <class 'function'>, <class 'NoneType'>

In [72]:
def metric_fn():
    return tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy',dtype=tf.float32)

In [73]:
metric = metric_fn()

In [70]:
def evaluate(eval_input_fn,model):
    eval_iter = iter(strategy.experimental_distribute_datasets_from_function(eval_input_fn))
    
    def _test_step_fn(inputs,label):
#         inputs,label = inputs
        model_outputs = model(inputs,training=False)
        metric.update_state(label,model_outputs)
    strategy.experimental_run_v2(_test_step_fn,args=(next(eval_iter)))

In [74]:
evaluate(eval_input_fn,trained_model)

In [78]:
metric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.125>

In [365]:
model_export_dir = '/var/extra/users/jgeorge/tf2.0/input/albert_base_custom/'

In [368]:
spm_model = os.path.join(model_dir, "assets", "30k-clean.model")
sp = spm.SentencePieceProcessor()
sp.load(spm_model)

True

In [369]:
trained_model.sp_model_file = tf.saved_model.Asset(spm_model)

In [373]:
trained_model.sp_model_file

In [374]:
trained_model.save(model_export_dir,include_optimizer=False,save_format='tf')

INFO:tensorflow:Assets written to: /var/extra/users/jgeorge/tf2.0/input/albert_base_custom/assets


INFO:tensorflow:Assets written to: /var/extra/users/jgeorge/tf2.0/input/albert_base_custom/assets


In [364]:
metric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.125>

In [345]:
bert_model = hub.KerasLayer(albert_hub_url,trainable=True,)

ValueError: Unknown signature default in https://tfhub.dev/google/albert_base/2 (available signatures: _SignatureMap({'tokenization_info': <tensorflow.python.eager.wrap_function.WrappedFunction object at 0x7f550be41450>, 'mlm': <tensorflow.python.eager.wrap_function.WrappedFunction object at 0x7f57ec038a50>, 'tokens': <tensorflow.python.eager.wrap_function.WrappedFunction object at 0x7f55086cf990>})).

In [94]:
bert_model = hub.KerasLayer(albert_hub_url,trainable=True,signature='tokens')

In [ ]:
bert_model()

In [126]:
trained_model = run_bert_classifier(
      strategy,
      bert_config,
      input_meta_data,
      albert_model_dir,
      epochs,
      steps_per_epoch,
      steps_per_loop,
      eval_steps,
      warmup_steps,
      learning_rate,
      init_checkpoint,
      train_input_fn,
      eval_input_fn,
      run_eagerly=False,
      use_keras_compile_fit=False)


TypeError: in converted code:

    /opt/custom/python/anaconda3/envs/tf2.1/lib/python3.7/site-packages/tensorflow_hub/keras_layer.py:209 call  *
        result = f()
    /opt/custom/python/anaconda3/envs/tf2.1/lib/python3.7/site-packages/tensorflow_core/python/eager/function.py:1551 __call__  *
        return self._call_impl(args, kwargs)
    /opt/custom/python/anaconda3/envs/tf2.1/lib/python3.7/site-packages/tensorflow_core/python/eager/function.py:1570 _call_impl
        ).format(self._num_positional_args, self._arg_keywords, args))

    TypeError: Expected at most 0 positional arguments (and the rest keywords, of ['segment_ids', 'input_ids', 'input_mask']), got ([<tf.Tensor 'segment_ids:0' shape=(None, 128) dtype=int32>, <tf.Tensor 'input_ids:0' shape=(None, 128) dtype=int32>, <tf.Tensor 'input_mask:0' shape=(None, 128) dtype=int32>],). When calling a concrete function, positional arguments may not be bound to Tensors within nested structures.


In [100]:
tags = set()
tags.add('train')

loaded_albert_model = tf.saved_model.load(albert_model_dir,tags=tags)

In [106]:
loaded_albert_model.signatures

_SignatureMap({'mlm': <tensorflow.python.eager.wrap_function.WrappedFunction object at 0x7f551de1d9d0>, 'tokens': <tensorflow.python.eager.wrap_function.WrappedFunction object at 0x7f551d98e090>, 'tokenization_info': <tensorflow.python.eager.wrap_function.WrappedFunction object at 0x7f551d5d5050>})

In [105]:
loaded_albert_model.signatures['mlm']

In [ ]:
# @tf.keras.utils.register_keras_serializable(package="Text")
#keeping one with print statements
class SentencepieceTokenization2(tf.keras.layers.Layer):
#     def __init__(self,model_proto):
    def __init__(self,model_path):
#         super(SentencepieceTokenization, self).__init__(trainable=False,dynamic=True,dtype=tf.int32)
        super(SentencepieceTokenization, self).__init__(trainable=False,dtype=tf.int32)
        #     self.sp_model_proto = model_proto
#         self.model_path = tf.saved_model.Asset(model_path)
        self.model_path = model_path
        
#         model_proto = tf.io.gfile.GFile(self.model_path, 'rb').read()
#         self.sp_model = tftext.SentencepieceTokenizer(model=model_proto)
    def build(self,input_shape):
#         self.model_proto = tf.io.gfile.GFile(self.model_path.asset_path, 'rb').read()
#         self.model_proto = tf.io.gfile.GFile(self.model_path, 'rb').read()
#         self.sp_model = tftext.SentencepieceTokenizer(model=self.model_proto)
        ## pad_id is usually 0 
        [self.CLS_ID,self.SEP_ID,self.PAD_ID]  = tfs.piece_to_id(['[CLS]','[SEP]','<pad>'],model_file=self.model_path)
        self.built=True
        
#     @tf.function    
    def call(self, input_text):
#         encoded_text = self.sp_model.tokenize(input_text).to_tensor()
        ##tensorflow sentence piece works while exporting to graph while, tf_text sentencepiece doesn't
#         encoded_text = tfs.encode(input_text,model_proto=self.model_proto)
        model_proto = tf.io.gfile.GFile(self.model_path, 'rb').read()
        encoded_text = self.get_encoded_text(input_text,model_proto=model_proto,max_sequence_length=20)
        
#         encoded_text = tf.RaggedTensor.from_tensor(encoded_text.values)
#         return encoded_text.to_tensor()        
        return encoded_text


    def get_encoded_text(self,input_text_batch,model_proto,max_sequence_length):
#         tf.print("input_text_batch is ",input_text_batch)
#         tf.print("type of input_text_batch ",type(input_text_batch))

        def process_invidual_line_encoding(x):
#             tf sentencepiece requires a list as input, while the individual value that we get here
#             is a single sentence, so adding one more dimension (i.e adding batch dimension = 1)
            list_x = tf.expand_dims(x,axis=0)
            sp_encoded = tfs.encode(list_x,model_proto=model_proto)
    #         removing the batch dim with size=1 (which we added in the previous step)
            values = tf.squeeze(sp_encoded.values,name='squeezed_values')
            sequence_length = tf.squeeze(sp_encoded.sequence_length)
#             tf.print('squeezed_values ' ,values)
    #         trimming to max_length-2 (-2 to incorporate [CLS], [SEP])
            trimmed_max_length = max_sequence_length-2
    #         values_trimmed = tf.slice(values,begin=[0],size=[trimmed_max_length],name='trimmed_out')
            values_trimmed = tf.cond(tf.greater(sequence_length,trimmed_max_length), 
                                    lambda : tf.slice(values,begin=[0],size=[trimmed_max_length],name='trimmed_out'),lambda : values)
#             tf.print('values_trimmed ',values_trimmed)
            concat = tf.concat([[self.CLS_ID],values_trimmed,[self.SEP_ID]],axis=0,name='concat_out')
#             tf.print('concat_out ',concat)
#             tf.print('size of concat ',tf.size(concat))
#             tf.print('shape of concat ',tf.shape(concat))
    #         actual_token_length = tf.size(concat) #this would also work since we are processing line by line in this function
            # tf.shape dynamic shape & variable.shape = static shape with dynamic entries = None
            actual_token_length = tf.shape(concat)[-1]
    #         need not prepend anything so 0 for 1st entry in padding, 
    #         & next value for padding is how many dimensions required at the end of tensor
            padded = tf.pad(concat,paddings=[[0,max_sequence_length-actual_token_length]],name='input_ids')
    #         segment_ids = tf.zeros(shape=tf.shape(padded),dtype=tf.int32)
    #         or
            segment_ids = tf.zeros_like(padded,dtype=tf.int32,name='segment_ids')

            input_mask = tf.scatter_nd(indices=tf.expand_dims(tf.range(0,actual_token_length),axis=1),
                                       updates=tf.ones(shape=[actual_token_length],dtype=tf.int32),
                                       shape=[max_sequence_length],name='input_mask')
            return (padded,segment_ids,input_mask)
    #     Issue running map_fn on gpu https://github.com/tensorflow/tensorflow/issues/28007 
    #     https://www.tensorflow.org/api_docs/python/tf/device
        with tf.device('/device:CPU:0'):
            encoded = tf.map_fn(lambda x: process_invidual_line_encoding(x),input_text_batch,dtype=(tf.int32,tf.int32,tf.int32))
#         tf.print('encoded map_fn ',encoded)
#         tf.print('encoded map_fn type ',type(encoded))
#         tf.print('encoded map_fn shape ',tf.shape(encoded))
    #     stacked = tf.stack(encoded)
        stacked = encoded
#         tf.print('stacked size ',tf.size(stacked))
#         tf.print('stacked shape ',tf.shape(stacked))
        return stacked
    
    
    def get_config(self):
        config = super(SentencepieceTokenization, self).get_config()
        config.update({'model_path': self.model_path})
        return config


In [43]:
[CLS_ID,SEP_ID]  = tfs.piece_to_id(['[CLS]','[SEP]'],model_proto=sp_model_proto)

In [239]:
CLS_ID

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [67]:
tfs.piece_to_id(['[CLS]','[SEP]','<pad>'],model_proto=sp_model_proto)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 3, 0], dtype=int32)>

In [252]:
tf.expand_dims(CLS_ID,axis=0)

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>

In [254]:
tf.repeat(CLS_ID,repeats=2)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 2], dtype=int32)>

In [261]:
tf.expand_dims(tf.repeat(CLS_ID,repeats=tf.shape(encoded.values)[0]),axis=1)

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[2],
       [2]], dtype=int32)>

In [260]:
tf.broadcast_to(CLS_ID,(tf.shape(encoded.values)[0],1))

<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
array([[2],
       [2]], dtype=int32)>

In [259]:
tf.shape(encoded.values)[0]

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [ ]:
tf.tile()
tf.stack()
tf.repeat()

In [256]:
tf.concat([encoded.values,tf.expand_dims(tf.repeat(CLS_ID,repeats=2),axis=1)],axis=1)

<tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[4148,   48,   25,   21, 1289, 5123,    2],
       [ 328, 1289, 5123,    0,    0,    0,    2]], dtype=int32)>

In [301]:
encoded = tfs.encode(['hi this is a test sentence','next test sentence'],model_proto=sp_model_proto)
encoded

SentencepieceEncodeDense(values=<tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[4148,   48,   25,   21, 1289, 5123],
       [ 328, 1289, 5123,    0,    0,    0]], dtype=int32)>, sequence_length=<tf.Tensor: shape=(2,), dtype=int32, numpy=array([6, 3], dtype=int32)>)

In [237]:
encoded.values

<tf.Tensor: shape=(1, 6), dtype=int32, numpy=array([[4148,   48,   25,   21, 1289, 5123]], dtype=int32)>

In [ ]:
encoded_single = tfs.encode(['next test sentence'],model_proto=sp_model_proto)
tf.pad()

In [310]:
sample_sentences = ['hi this is a test sentence','next test sentence']
e_sparse  = tfs.encode_sparse(sample_sentences,model_proto=sp_model_proto)

In [304]:
encoded.values

<tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[4148,   48,   25,   21, 1289, 5123],
       [ 328, 1289, 5123,    0,    0,    0]], dtype=int32)>

In [307]:
tfs.id_to_piece(encoded.values.numpy(),model_proto=sp_model_proto)

<tf.Tensor: shape=(2, 6), dtype=string, numpy=
array([[b'\xe2\x96\x81hi', b'\xe2\x96\x81this', b'\xe2\x96\x81is',
        b'\xe2\x96\x81a', b'\xe2\x96\x81test', b'\xe2\x96\x81sentence'],
       [b'\xe2\x96\x81next', b'\xe2\x96\x81test',
        b'\xe2\x96\x81sentence', b'<pad>', b'<pad>', b'<pad>']],
      dtype=object)>

In [311]:
sp3 = tftext.SentencepieceTokenizer(model=sp_model_proto)
encoded_text3 = sp3.tokenize(sample_sentences)
encoded_text3

<tf.RaggedTensor [[4148, 48, 25, 21, 1289, 5123], [328, 1289, 5123]]>

In [68]:
encoded = tfs.encode(['hi this is a test sentence','next test sentence'],model_proto=sp_model_proto)
encoded

SentencepieceEncodeDense(values=<tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[4148,   48,   25,   21, 1289, 5123],
       [ 328, 1289, 5123,    0,    0,    0]], dtype=int32)>, sequence_length=<tf.Tensor: shape=(2,), dtype=int32, numpy=array([6, 3], dtype=int32)>)

In [77]:
tf.split(encoded.values,num_or_size_splits=encoded.values.shape[0],axis=0)

[<tf.Tensor: shape=(1, 6), dtype=int32, numpy=array([[4148,   48,   25,   21, 1289, 5123]], dtype=int32)>,
 <tf.Tensor: shape=(1, 6), dtype=int32, numpy=array([[ 328, 1289, 5123,    0,    0,    0]], dtype=int32)>]

In [273]:
concatenated_values(CLS_ID,encoded.values,SEP_ID)

<tf.Tensor: shape=(2, 8), dtype=int32, numpy=
array([[   2, 4148,   48,   25,   21, 1289, 5123,    3],
       [   2,  328, 1289, 5123,    0,    0,    0,    3]], dtype=int32)>

In [277]:
def concatenated_values(CLS_ID,encoded_values,SEP_ID):
    batch_size = encoded_values.shape[0]
#     Note that we are not using tf.shape to get the shape as it doesn't get the runtime shape
#     batch_size = tf.shape(encoded_values)[0]
    ##we need the CLS_ID to be prepended & SEP_ID to be appended to encoded_values from sentence_piece
    ## the encoded_values have shape = (batch_size,sequence_length)
    
    ## to concat values we need all dimension except the ones where are concat is happening to be the same
    ## here it is 2d with 1st dimension being batch_size, so getting tensors of shape (batch_size,1)
    CLS_IDS = tf.broadcast_to(CLS_ID,(batch_size,1))
    SEP_IDS = tf.broadcast_to(SEP_ID,(batch_size,1))
#     or another way to get shape = (batch_size,1)
#     CLS_IDS = tf.expand_dims(tf.repeat(CLS_ID,repeats=batch_size),axis=1)
#     SEP_IDS = tf.expand_dims(tf.repeat(SEP_ID,repeats=batch_size),axis=1)

    return tf.concat([CLS_IDS,encoded_values,SEP_IDS],axis=1)

In [278]:
concatenated_values(CLS_ID,encoded.values,SEP_ID)

<tf.Tensor: shape=(2, 8), dtype=int32, numpy=
array([[   2, 4148,   48,   25,   21, 1289, 5123,    3],
       [   2,  328, 1289, 5123,    0,    0,    0,    3]], dtype=int32)>

In [274]:
a = tf.keras.layers.Input((None, 10))
a

<tf.Tensor 'input_1:0' shape=(None, None, 10) dtype=float32>

In [275]:
tf.shape(a)

<tf.Tensor 'Shape:0' shape=(3,) dtype=int32>

In [276]:
a.shape

TensorShape([None, None, 10])